In [1]:
import numpy as np
import pandas as pd
import time
from datetime import datetime, timezone
from scipy.optimize import minimize, least_squares
import yfinance as yf
from scipy.integrate import quad

# --- Heston Pricing Functions ---
def heston_characteristic_function(phi, S, K, T, r, kappa, rho, volvol, theta, var0, div, P1P2):
    x = np.log(S)
    a = kappa * theta
    u = 0.5 if P1P2 == 1 else -0.5
    b = kappa - rho * volvol if P1P2 == 1 else kappa
    d = np.sqrt((rho * volvol * phi * 1j - b)**2 - volvol**2 * (2 * u * phi * 1j - phi**2))
    g = (b - rho * volvol * phi * 1j + d) / (b - rho * volvol * phi * 1j - d)
    C = (r - div) * phi * 1j * T + (a / volvol**2) * ((b - rho * volvol * phi * 1j + d) * T - 2 * np.log((1 - g * np.exp(d * T)) / (1 - g)))
    D = (b - rho * volvol * phi * 1j + d) / volvol**2 * (1 - np.exp(d * T)) / (1 - g * np.exp(d * T))
    return np.exp(C + D * var0 + 1j * phi * x)

def heston_call_price(S, K, T, r, kappa, rho, volvol, theta, var0, div):
    def integrand(phi, P1P2):
        cf = heston_characteristic_function(phi, S, K, T, r, kappa, rho, volvol, theta, var0, div, P1P2)
        return np.real(np.exp(-1j * phi * np.log(K)) * cf / (1j * phi))

    eps = 1e-6           # lower integration limit (avoids φ = 0)
    integral_P1 = quad(lambda phi: integrand(phi, 1),
                    eps, 100, limit=200, epsabs=1e-8, epsrel=1e-8)[0]
    integral_P2 = quad(lambda phi: integrand(phi, 2),
                    eps, 100, limit=200, epsabs=1e-8, epsrel=1e-8)[0]
    
    P1 = 0.5 + (1 / np.pi) * integral_P1
    P2 = 0.5 + (1 / np.pi) * integral_P2
    return max(0.0, S * np.exp(-div * T) * P1 - K * np.exp(-r * T) * P2)

def heston_put_price(S, K, T, r, kappa, rho, volvol, theta, var0, div):
    """Put option pricing function using put-call parity"""
    CallValue = heston_call_price(S, K, T, r, kappa, rho, volvol, theta, var0, div)
    PutValue = CallValue - S * np.exp(-div * T) + K * np.exp(-r * T)
    return PutValue

def heston_prices_parallel(params, Spots, Strikes, Maturities, Rates, div):
    kappa, rho, volvol, theta, var0 = params
    return np.array([
        heston_call_price(S, K, T, r, kappa, rho, volvol, theta, var0, div)
        for S, K, T, r in zip(Spots, Strikes, Maturities, Rates)
    ])

def OptFunctionFast(params, Spots, Maturities, Rates, Strikes, MarketP, div, check_bounds=True):
    kappa, rho, volvol, theta, var0 = params
    if check_bounds and not (0.1 <= kappa <= 15.0 and -0.99 <= rho <= 0.0 and 0.01 <= volvol <= 2.0 and 0.001 <= theta <= 0.5 and 0.001 <= var0 <= 0.5):
        return 1e10

    valid = np.isfinite(MarketP) & (MarketP > 0)
    if not np.any(valid):
        return 1e10

    Spots, Strikes, Maturities, Rates, MarketP = Spots[valid], Strikes[valid], Maturities[valid], Rates[valid], MarketP[valid]
    model_prices = heston_prices_parallel(params, Spots, Strikes, Maturities, Rates, div)

    abs_errors = np.abs(model_prices - MarketP)
    if np.mean(abs_errors) > 10.0:
        return np.mean(abs_errors) * 10

    moneyness = Strikes / Spots
    weights = 1.0 + np.exp(-20.0 * (moneyness - 1.0)**2)
    weighted_errors = (model_prices - MarketP)**2 * weights

    return np.mean(weighted_errors) if np.isfinite(np.mean(weighted_errors)) else 1e10


In [63]:
def get_clean_market_data(symbol, expiration, s0, max_options=50, min_maturity_days=5):
    tk = yf.Ticker(symbol)
    all_expirations = pd.to_datetime(tk.options).date
    tgt_date = pd.to_datetime(expiration).date()
    today = datetime.now(timezone.utc).date()
    
    i = np.searchsorted(all_expirations, tgt_date)
    selected_exps = all_expirations[max(0, i-3):i+4]  # pick expiries before/after

    parts = []
    for expiry in selected_exps:
        chain = tk.option_chain(expiry.isoformat()).calls
        chain = chain.dropna(subset=["bid", "ask", "impliedVolatility", "volume", "openInterest"])
        chain = chain[(chain.bid > 0) & (chain.ask > 0)]

        mid = (chain.bid + chain.ask) / 2.0
        spread_pct = (chain.ask - chain.bid) / mid * 100
        maturity = (expiry - today).days / 365.25
        moneyness = chain.strike / s0

        df = chain.assign(mid=mid, spread_pct=spread_pct, maturity=maturity, moneyness=moneyness)
        df = df[
            (df.spread_pct < 10.0) &
            (0.85 <= df.moneyness) & (df.moneyness <= 1.15) &
            (df.maturity > min_maturity_days / 365.25)
        ]

        if not df.empty:
            parts.append(df[["strike", "mid", "maturity", "impliedVolatility"]])

    if not parts:
        return pd.DataFrame()

    df = pd.concat(parts, ignore_index=True)
    
    # NEW: Group options by expiry maturity buckets
    df = df.sort_values(by=["maturity", "strike"]).reset_index(drop=True)

    # Limit to max_options but **diversify maturities**:
    unique_maturities = df.maturity.unique()
    selected = []

    for mat in unique_maturities:
        subset = df[df.maturity == mat]
        n_select = min(len(subset), max(1, max_options // len(unique_maturities)))
        selected.append(subset.head(n_select))

    final_df = pd.concat(selected).sort_values(by=["maturity", "strike"]).reset_index(drop=True)
    return final_df


In [2]:
def get_clean_market_data(symbol, expiration, s0, max_options=50, min_maturity_days=5):
    tk = yf.Ticker(symbol)
    all_expirations = pd.to_datetime(tk.options).date
    tgt_date = pd.to_datetime(expiration).date()
    today = datetime.now(timezone.utc).date()

    # Find the exact expiration
    if tgt_date not in all_expirations:
        print(f"Warning: Target expiration {tgt_date} not found. Available expirations: {all_expirations}")
        return pd.DataFrame()

    try:
        chain = tk.option_chain(tgt_date.isoformat()).calls
        chain = chain.dropna(subset=["bid", "ask", "impliedVolatility", "volume", "openInterest"])
        chain = chain[(chain.bid > 0) & (chain.ask > 0)]

        mid = (chain.bid + chain.ask) / 2.0
        spread_pct = (chain.ask - chain.bid) / mid * 100
        maturity = (tgt_date - today).days / 365.25
        moneyness = chain.strike / s0

        df = chain.assign(mid=mid, spread_pct=spread_pct, maturity=maturity, moneyness=moneyness)

        df = df[
            (df.spread_pct < 10.0) &
            (0.85 <= df.moneyness) & (df.moneyness <= 1.15) &
            (df.maturity > min_maturity_days / 365.25)
        ]

        if df.empty:
            return pd.DataFrame()

        # Sort strikes nicely
        df = df.sort_values(by=["strike"]).reset_index(drop=True)

        # Pick up to max_options
        df = df.head(max_options)

        return df[["strike", "mid", "maturity", "impliedVolatility"]]

    except Exception as e:
        print(f"Failed to load option chain: {e}")
        return pd.DataFrame()


In [3]:
# --- Calibration Function ---
def calibrate_heston_multi_optimizers(symbol, expiration, s0, r,
                                      div, max_options, max_time=300,
                                      methods=None):
    """
    Calibrate Heston with a list of optimiser names.

    Parameters
    ----------
    methods : list[str] or None
        Names understood by either `scipy.optimize.minimize` **or**
        the special tag 'LM' (Levenberg-Marquardt via `least_squares`) and
        'DE' (global Differential Evolution).

        Default uses a broad, sensible set.
    """
    if methods is None:
        methods = [
            # local gradient-based
            "L-BFGS-B",
            # constrained gradient
            "SLSQP",
            # Levenberg-Marquardt (least_squares, NO bounds)
            "LM",
        ]

    start = time.time()
    
    data = get_clean_market_data(symbol, expiration, s0, max_options , min_maturity_days=5)
    
    print(f"Data loaded in {time.time() - start:.2f} seconds")
    print(f"Market data used: {len(data)} options")
    
    if data.empty:
        return {"success": False, "error": "No data found"}

    Spots    = np.full_like(data.maturity.values, s0, dtype=float)
    Strikes  = data.strike.values
    Maturities = data.maturity.values
    Rates    = np.full_like(data.maturity.values, r, dtype=float)
    MarketP  = data.mid.values

    avg_iv   = np.mean(data.impliedVolatility)
    init_var = avg_iv ** 2
    initial_guess = [1.5, -0.7, 0.3 * avg_iv, init_var, init_var]
    bounds = [(0.1, 10.0), (-0.95, 0.0), (0.01, 1.5),
              (0.001, 0.4), (0.001, 0.4)]

    results = []
    for method in methods:
        if time.time() - start > max_time:
            break
        try:
            t0 = time.time()  # <--- start timing this method
            if method == "LM":
                def residuals(p, Spots, Strikes, Mats, Rates, Market, div):
                    kappa, rho, volvol, theta, var0 = p
                    if not (0.1 <= kappa <= 15.0 and -0.99 <= rho <= 0.0
                            and 0.01 <= volvol <= 2.0 and 0.001 <= theta <= 0.5
                            and 0.001 <= var0 <= 0.5):
                        return np.full_like(Market, 1e5)
                    model = heston_prices_parallel(p, Spots, Strikes, Mats, Rates, div)
                    return model - Market

                res = least_squares(
                    residuals, initial_guess, method='lm',
                    args=(Spots, Strikes, Maturities, Rates, MarketP, div),
                    xtol=1e-12, ftol=1e-12, gtol=1e-12)
                xopt = res.x
                model_prices = heston_prices_parallel(
                    xopt, Spots, Strikes, Maturities, Rates, div)

            else:
                res = minimize(
                    OptFunctionFast, initial_guess, method=method, bounds=bounds,
                    args=(Spots, Maturities, Rates, Strikes, MarketP, div, True),
                    options={"maxiter": 500, "disp": False})
                xopt = res.x
                model_prices = heston_prices_parallel(
                    xopt, Spots, Strikes, Maturities, Rates, div)

            elapsed = time.time() - t0  # <--- end timing this method
            mse = np.mean((model_prices - MarketP) ** 2)
            results.append((method, mse, xopt, elapsed))
            print(f"Method: {method:11s} | MSE: {mse:.6f} | Time: {elapsed:.2f} s")
        except Exception as err:
            print(f"Method: {method:11s} failed → {err}")
            continue

    if not results:
        return {"success": False, "error": "All methods failed"}

    best_method, best_mse, best_params, best_time = min(results, key=lambda t: t[1])

    
    return {
        "success": True,
        "best_method": best_method,
        "mse": best_mse,
        "parameters": dict(zip(["kappa", "rho", "volvol", "theta", "var0"],
                               best_params)),
        "market_data_used": len(data),
        "calibration_time": time.time() - start,
        "all_results": [
            {"method": m, "mse": e, "params": p.tolist(), "time": t}
            for m, e, p, t in results
        ]
    }

In [7]:
import pandas as pd

# --- Calibration Function ---
def calibrate_heston_multi_optimizers(symbol, expiration, s0, r,
                                      div, max_options, max_time=300,
                                      methods=None,
                                      save_excel_path=None):
    if methods is None:
        methods = ["L-BFGS-B", "SLSQP", "LM"]

    start = time.time()
    
    data = get_clean_market_data(symbol, expiration, s0, max_options, min_maturity_days=5)
    
    print(f"Data loaded in {time.time() - start:.2f} seconds")
    print(f"Market data used: {len(data)} options")
    
    if data.empty:
        return {"success": False, "error": "No data found"}

    Spots    = np.full_like(data.maturity.values, s0, dtype=float)
    Strikes  = data.strike.values
    Maturities = data.maturity.values
    Rates    = np.full_like(data.maturity.values, r, dtype=float)
    MarketP  = data.mid.values

    avg_iv   = np.mean(data.impliedVolatility)
    init_var = avg_iv ** 2
    initial_guess = [1.5, -0.7, 0.3 * avg_iv, init_var, init_var]
    bounds = [(0.1, 10.0), (-0.95, 0.0), (0.01, 1.5),
              (0.001, 0.4), (0.001, 0.4)]

    results = []
    detailed_results = []

    for method in methods:
        if time.time() - start > max_time:
            break
        try:
            t0 = time.time()
            if method == "LM":
                def residuals(p, Spots, Strikes, Mats, Rates, Market, div):
                    kappa, rho, volvol, theta, var0 = p
                    if not (0.1 <= kappa <= 15.0 and -0.99 <= rho <= 0.0
                            and 0.01 <= volvol <= 2.0 and 0.001 <= theta <= 0.5
                            and 0.001 <= var0 <= 0.5):
                        return np.full_like(Market, 1e5)
                    model = heston_prices_parallel(p, Spots, Strikes, Mats, Rates, div)
                    return model - Market

                res = least_squares(
                    residuals, initial_guess, method='lm',
                    args=(Spots, Strikes, Maturities, Rates, MarketP, div),
                    xtol=1e-12, ftol=1e-12, gtol=1e-12)
                xopt = res.x
                model_prices = heston_prices_parallel(
                    xopt, Spots, Strikes, Maturities, Rates, div)

            else:
                res = minimize(
                    OptFunctionFast, initial_guess, method=method, bounds=bounds,
                    args=(Spots, Maturities, Rates, Strikes, MarketP, div, True),
                    options={"maxiter": 500, "disp": False})
                xopt = res.x
                model_prices = heston_prices_parallel(
                    xopt, Spots, Strikes, Maturities, Rates, div)

            elapsed = time.time() - t0
            total_mse = np.mean((model_prices - MarketP) ** 2)
            per_option_mse = (model_prices - MarketP) ** 2

            # save full details
            detailed_results.append({
                "method": method,
                "total_mse": total_mse,
                "time": elapsed,
                "params": xopt,
                "model_prices": model_prices,
                "market_prices": MarketP,
                "per_option_mse": per_option_mse,
                "strikes": Strikes,
                "maturities": Maturities
            })

            results.append((method, total_mse, xopt, elapsed))
            print(f"Method: {method:11s} | Total MSE: {total_mse:.6f} | Time: {elapsed:.2f} s")

        except Exception as err:
            print(f"Method: {method:11s} failed → {err}")
            continue

    if not results:
        return {"success": False, "error": "All methods failed"}

    best_method, best_mse, best_params, best_time = min(results, key=lambda t: t[1])

    # --------- Save to Excel --------------
    if save_excel_path:
        writer = pd.ExcelWriter(save_excel_path, engine='xlsxwriter')

        for detail in detailed_results:
            df = pd.DataFrame({
                "Strike": detail["strikes"],
                "Maturity (years)": detail["maturities"],
                "Market Price": detail["market_prices"],
                "Model Price": detail["model_prices"],
                "Per Option MSE": detail["per_option_mse"],
                "Absolute Error": np.abs(detail["model_prices"] - detail["market_prices"]),
                "Relative Error (%)": 100 * np.abs(detail["model_prices"] - detail["market_prices"]) / detail["market_prices"]
            })

            sheet_name = detail["method"][:30]  # Excel max 31 chars
            df.to_excel(writer, sheet_name=sheet_name, index=False)

        # summary sheet
        summary = pd.DataFrame([{
            "Method": d["method"],
            "Total MSE": d["total_mse"],
            "Time (s)": d["time"],
            "Kappa": d["params"][0],
            "Rho": d["params"][1],
            "VolVol": d["params"][2],
            "Theta": d["params"][3],
            "V0": d["params"][4]
        } for d in detailed_results])
        summary.to_excel(writer, sheet_name="Summary", index=False)

        writer.close()  # Change from writer.save() to writer.close()
        print(f"Results saved to {save_excel_path}")

    # --------- Done --------------
    return {
        "success": True,
        "best_method": best_method,
        "mse": best_mse,
        "parameters": dict(zip(["kappa", "rho", "volvol", "theta", "var0"],
                               best_params)),
        "market_data_used": len(data),
        "calibration_time": time.time() - start,
        "all_results": [
            {"method": m, "mse": e, "params": p.tolist(), "time": t}
            for m, e, p, t in results
        ]
    }


In [9]:

# Calibrate
results = calibrate_heston_multi_optimizers(
    symbol='SPY',
    expiration='2025-05-09', 
    s0=550.64,
    r=0.0432,
    div=0.00,
    max_options=100,
    methods=["L-BFGS-B", "LM"],
    save_excel_path="heston_calibration_results.xlsx"
)
# Print Results
if results["success"]:
    print(f"Best method: {results['best_method']}")
    print(f"Best MSE: {results['mse']:.6f}")
    print("Calibrated Parameters:")
    for key, value in results["parameters"].items():
        print(f"  {key}: {value:.6f}")
    print(f"\nCalibration took {results['calibration_time']:.2f} seconds.")
    
    print("All methods results:")
    for res in results["all_results"]:
        print(f"  Method: {res['method']}, MSE: {res['mse']:.6f}, Time: {res['time']:.2f} s, Params: {res['params']}")

else:
    print("Calibration failed:", results["error"])


Data loaded in 0.87 seconds
Market data used: 94 options
Method: L-BFGS-B    | Total MSE: 0.081119 | Time: 47.04 s
Method: LM          | Total MSE: 0.035302 | Time: 30.79 s
Results saved to heston_calibration_results.xlsx
Best method: LM
Best MSE: 0.035302
Calibrated Parameters:
  kappa: 1.504879
  rho: -0.735431
  volvol: 2.000000
  theta: 0.217590
  var0: 0.068672

Calibration took 78.73 seconds.
All methods results:
  Method: L-BFGS-B, MSE: 0.081119, Time: 47.04 s, Params: [0.1, -0.8314524362461562, 1.5, 0.0010033026077214222, 0.06953696403522645]
  Method: LM, MSE: 0.035302, Time: 30.79 s, Params: [1.504879286516676, -0.7354306904407899, 1.9999999892303877, 0.2175896379520944, 0.06867233937814744]
